In [1]:
import platform
print(platform.python_version())
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

3.6.7


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3683715674207936907, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8137145047541298318
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14454833893249691046
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14800692839
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13401239721814633736
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!ls '/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code'

 cage_0.mp4	 faceA_0		    mtcnn_weights
 checkpoints	 faceB			    preprocess.py
 checkpoints_0	 faceB_0		    __pycache__
 converter	'Image Swap GAN(0).ipynb'   train_samples_a.pkl
 detector	'Image Swap GAN.ipynb'	    train_samples_b.pkl
 dummy.mp4	 lib			    trump_0.mp4
 faceA		 mtcnn_detect_face.py	    umeyama.py


In [4]:
%cd '/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code'

/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code


In [0]:
#!git clone https://github.com/shaoanlu/faceswap-GAN.git

In [0]:
%matplotlib inline

import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random as rand
import glob
import os
import math
from PIL import Image

In [7]:
%pwd

'/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code'

In [0]:
# from google.colab import files

In [0]:
# # Upload source video
# source_video = files.upload()

# for fn_source_video, _ in source_video.items():
#     print(fn_source_video)

In [0]:
# # Upload target video
# target_video = files.upload()

# for fn_target_video, _ in target_video.items():
#     print(fn_target_video)

In [0]:
# %%capture
# !pip install moviepy
# !pip install keras_vggface
# import imageio
# imageio.plugins.ffmpeg.download()

In [12]:
import keras.backend as K
from detector.face_detector import MTCNNFaceDetector
import glob

from preprocess import preprocess_video

Using TensorFlow backend.


In [0]:
# fd = MTCNNFaceDetector(sess=K.get_session(), model_path="./mtcnn_weights/")

In [0]:
# !mkdir -p faceA/rgb
# !mkdir -p faceA/binary_mask
# !mkdir -p faceB/rgb
# !mkdir -p faceB/binary_mask

In [0]:
fn_source_video = 'trump_0.mp4'
fn_target_video = 'cage_0.mp4'  
  
# save_interval1 = 5 # perform face detection every {save_interval} frames
# save_path = "./faceA/"
# preprocess_video(fn_source_video, fd, save_interval1, save_path)
# save_interval2 = 2
# save_path = "./faceB/"
# preprocess_video(fn_target_video, fd, save_interval2, save_path)

In [16]:
print(str(len(glob.glob("faceA/rgb/*.*"))) + " face(s) extracted from source video: " + fn_source_video + ".")
print(str(len(glob.glob("faceB/rgb/*.*"))) + " face(s) extracted from target video: " + fn_target_video + ".")

708 face(s) extracted from source video: trump_0.mp4.
647 face(s) extracted from target video: cage_0.mp4.


In [0]:
# Path to training images
img_dirA = './faceA/rgb'
img_dirB = './faceB/rgb'
img_dirA_bm_eyes = "./faceA/binary_mask"
img_dirB_bm_eyes = "./faceB/binary_mask"

In [18]:
# Get filenames
train_a = glob.glob(img_dirA+"/*.*")
train_b = glob.glob(img_dirB+"/*.*")

train_AnB = train_a + train_b

assert len(train_a), "No image found in " + str(img_dirA)
assert len(train_b), "No image found in " + str(img_dirB)
print ("Number of images in folder A: " + str(len(train_a)))
print ("Number of images in folder B: " + str(len(train_b)))

Number of images in folder A: 708
Number of images in folder B: 647


In [19]:
# train_A = train_a[:1000]
# train_B = train_b[:1000]
print ("Number of images in training A: " + str(len(train_a)))
print ("Number of images in training B: " + str(len(train_b)))

Number of images in training A: 708
Number of images in training B: 647


In [0]:
def model_inputs(dim):
    inputs_real = tf.placeholder(tf.float32, (None, dim), name='input_real') 
    inputs_fake = tf.placeholder(tf.float32, (None, dim), name='input_fake')
    
    return inputs_real, inputs_fake

In [0]:
def preprocess_image(original_image):
    maximum = np.max(original_image)
    minimum = np.min(original_image)
    # Scale to 0 and 1
    processed_image = (original_image - minimum)/(maximum-minimum)
    # Scale between -1 and 1
    processed_image = (processed_image*2) - 1
    return processed_image
    
def postprocess_image(original_image, processed_image):
    maximum = np.max(original_image)
    minimum = np.min(original_image)
    proc_max = np.max(processed_image)
    proc_min = np.min(processed_image)
    # Scale between -1 and 1
    output_image = (processed_image - proc_min)/(proc_max - proc_min)
    # Scale to 0 and 1
    output_image = (output_image * (maximum-minimum)) + minimum
    output_image = output_image.astype(int)
    return output_image

In [0]:
# def encoder(image_input, latent_size, n_units=128, reuse=False, alpha=0.01, scope='encoder'):
#     with tf.variable_scope(scope, reuse=reuse):
        
#         # Hidden layer
#         hidden = tf.layers.dense(image_input, n_units, activation=None)
#         print('hidden is:', hidden)
        
#         # Leaky ReLU
#         hidden = tf.maximum(hidden, alpha*hidden)
#         print('hidden is:', hidden)
        
#         # Logits and tanh output
#         logits = tf.layers.dense(hidden, latent_size, activation=None)
#         print('logits is:', logits)
#         latents = tf.nn.tanh(logits)
#         print('latents out is:', latents)
#         return latents
    
# def decoder(latents, out_dim, n_units=128, reuse=False, alpha=0.01, scope='decoder'):
#     with tf.variable_scope(scope, reuse=reuse):
        
#         # Hidden layer
#         hidden = tf.layers.dense(latents, n_units, activation=None)
#         print('hidden is:', hidden)
        
#         # Leaky ReLU
#         hidden = tf.maximum(hidden, alpha*hidden)
#         print('hidden is:', hidden)

#         # Logits and tanh output
#         logits = tf.layers.dense(hidden, out_dim, activation=None)
#         print('logits is:', logits)
#         out = tf.nn.tanh(logits)
#         print('decoder out is:', out)
        
#         return out
    
# def discriminator(inputs, n_units=128, reuse=False, alpha=0.01, scope='discriminator'):
#     with tf.variable_scope(scope, reuse=reuse):
        
#         # Hidden layer
#         h1 = tf.layers.dense(inputs, n_units, activation=None)
#         print('hidden is:', h1)
        
#         # Leaky ReLU
#         h1 = tf.maximum(alpha * h1, h1)
#         print('hidden is:', h1)

#         logits = tf.layers.dense(h1, 1, activation=None)
#         print('logits is:', logits)
#         out = tf.sigmoid(logits)
#         print('discriminator out is:', out)
        
#         return out, logits

In [0]:
# losses da,ga,db,gb saved as: (0.9020894, 3.3791414e-06, 0.36076805, 0.23865208)
# batch_a (100, 12288)
# batch_b (100, 12288)
# encoder_val Tensor("generator_1/Tanh:0", shape=(?, 100), dtype=float32)
# decoder_a_val Tensor("dec_a_1/Tanh:0", shape=(?, 12288), dtype=float32)
# feed_dict_decoder {<tf.Tensor 'input_fake:0' shape=(?, 12288) dtype=float32>: array([[-0.20784314, -0.36470588, -0.56078431, ..., -0.71764706,
#         -0.80392157, -0.85098039],
#        [-0.56862745, -0.62352941, -0.68627451, ...,  0.23921569,
#         -0.04313725, -0.16862745],
#        [-0.56078431, -0.63137255, -0.68627451, ...,  0.08235294,
#         -0.29411765, -0.4745098 ],
#        ...,
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.09803922,
#         -0.34117647, -0.4745098 ],
#        [-0.56078431, -0.61568627, -0.67843137, ...,  0.09803922,
#         -0.30980392, -0.4745098 ],
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.56862745,
#          0.27843137,  0.12941176]])}
# samples_a saved as: []
# samples_b saved as: []

# losses da,ga,db,gb saved as: (0.5098937, 11.09968, 0.43515426, 14.043441)
# batch_a (100, 12288)
# batch_b (100, 12288)
# inside encoder
# input_ shape is: (?, 64, 64, 3)
# x shape is: (?, 64, 64, 3)
# x shape is: (?, 32, 32, 128)
# x shape is: (?, 16, 16, 256)
# x shape is: (?, 8, 8, 512)
# x shape is: (?, 4, 4, 1024)
# x shape is: (?, 1024)
# x shape is: (?, 16384)
# x shape is: (?, 4, 4, 1024)
# x shape is: (?, 8, 8, 512)
# encoder_val Tensor("generator_1/pixel_shuffler_8/Reshape_1:0", shape=(?, 8, 8, 512), dtype=float32)
# inside decoder
# input_ shape is: (?, 8, 8, 512)
# x shape is: (?, 8, 8, 512)
# x shape is: (?, 16, 16, 256)
# x shape is: (?, 32, 32, 128)
# x shape is: (?, 64, 64, 64)
# x shape is: (?, 64, 64, 3)
# will leave the decoder, x is :  Tensor("dec_a_1/conv2d_22/Sigmoid:0", shape=(?, 64, 64, 3), dtype=float32)
# decoder_a_val Tensor("dec_a_1/conv2d_22/Sigmoid:0", shape=(?, 64, 64, 3), dtype=float32)
# feed_dict_decoder {<tf.Tensor 'input_fake:0' shape=(?, 12288) dtype=float32>: array([[-0.56078431, -0.63137255, -0.68627451, ...,  0.23921569,
#         -0.09803922, -0.23137255],
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.17647059,
#         -0.11372549, -0.23921569],
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.11372549,
#         -0.30196078, -0.44313725],
#        ...,
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.14509804,
#         -0.04313725, -0.12156863],
#        [-0.67058824, -0.77254902, -0.90588235, ..., -0.37254902,
#         -0.54509804, -0.65490196],
#        [ 0.20784314,  0.01960784, -0.2       , ..., -0.70980392,
#         -0.78823529, -0.88235294]])}


# losses da,ga,db,gb saved as: (0.5789854, 3.9114652, 1.0304557, 472.15677)
# batch_a (100, 12288)
# batch_b (100, 12288)
# inside encoder
# input_ shape is: (?, 64, 64, 3)
# x shape is: (?, 64, 64, 3)
# x shape is: (?, 32, 32, 128)
# x shape is: (?, 16, 16, 256)
# x shape is: (?, 8, 8, 512)
# x shape is: (?, 4, 4, 1024)
# x shape is: (?, 1024)
# x shape is: (?, 16384)
# x shape is: (?, 4, 4, 1024)
# x shape is: (?, 8, 8, 512)
# encoder_val Tensor("generator_1/pixel_shuffler_8/Reshape_1:0", shape=(?, 8, 8, 512), dtype=float32)
# inside decoder
# input_ shape is: (?, 8, 8, 512)
# x shape is: (?, 8, 8, 512)
# x shape is: (?, 16, 16, 256)
# x shape is: (?, 32, 32, 128)
# x shape is: (?, 64, 64, 64)
# x shape is: (?, 64, 64, 3)
# x shape is: (?, 12288)
# x shape is: (?, 12288)
# x shape is: (?, 64, 64, 3)
# decoder_a_val Tensor("dec_a_1/reshape_5/Reshape:0", shape=(?, 64, 64, 3), dtype=float32)
# feed_dict_decoder {<tf.Tensor 'input_fake:0' shape=(?, 12288) dtype=float32>: array([[-0.56078431, -0.63137255, -0.68627451, ...,  0.55294118,
#          0.20784314,  0.01176471],
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.12941176,
#         -0.30980392, -0.44313725],
#        [-0.57647059, -0.64705882, -0.70196078, ...,  0.09019608,
#         -0.28627451, -0.45882353],
#        ...,
#        [-0.56862745, -0.65490196, -0.68627451, ...,  0.08235294,
#         -0.29411765, -0.45098039],
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.11372549,
#         -0.30196078, -0.44313725],
#        [-0.56078431, -0.64705882, -0.69411765, ...,  0.1372549 ,
#         -0.30980392, -0.44313725]])}

# FailedPreconditionError: Attempting to use uninitialized value dec_a_1/dense_7/bias
# 	 [[{{node dec_a_1/dense_7/bias/read}}]]

In [0]:
from keras.layers import Input, Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from lib.PixelShuffler import PixelShuffler

In [0]:
IMAGE_SHAPE = (64, 64, 3)
ENCODER_DIM = 1024

In [0]:
def conv_block(x,filters):
            x = Conv2D(filters, kernel_size=5, strides=2, padding='same')(x)
            x = LeakyReLU(0.1)(x)
            return x
      
def upscale_block(x,filters):
            x = Conv2D(filters * 4, kernel_size=3, padding='same')(x)
            x = LeakyReLU(0.1)(x) # 使用 LeakyReLU 激活函数
            x = PixelShuffler()(x) # 将filter的大小变为原来的1/4，让高和宽变为原来的两倍
            return x
def lrelu(x):
    return tf.maximum(x, tf.multiply(x, 0.2))

In [0]:
# def encoder(image_input, latent_size, n_units=128, reuse=False, alpha=0.01, scope='encoder'):
#     with tf.variable_scope(scope, reuse=reuse):
        
#         # Hidden layer
#         hidden = tf.layers.dense(image_input, n_units, activation=None)
#         print('hidden is:', hidden)
        
#         # Leaky ReLU
#         hidden = tf.maximum(hidden, alpha*hidden)
#         print('hidden is:', hidden)
        
#         # Logits and tanh output
#         logits = tf.layers.dense(hidden, latent_size, activation=None)
#         print('logits is:', logits)
#         latents = tf.nn.tanh(logits)
#         print('latents out is:', latents)
#         return latents
    
# def decoder(latents, out_dim, n_units=128, reuse=False, alpha=0.01, scope='decoder'):
#     with tf.variable_scope(scope, reuse=reuse):
        
#         # Hidden layer
#         hidden = tf.layers.dense(latents, n_units, activation=None)
#         print('hidden is:', hidden)
        
#         # Leaky ReLU
#         hidden = tf.maximum(hidden, alpha*hidden)
#         print('hidden is:', hidden)

#         # Logits and tanh output
#         logits = tf.layers.dense(hidden, out_dim, activation=None)
#         print('logits is:', logits)
#         out = tf.nn.tanh(logits)
#         print('decoder out is:', out)
        
#         return out
    
# def discriminator(inputs, n_units=128, reuse=False, alpha=0.01, scope='discriminator'):
#     with tf.variable_scope(scope, reuse=reuse):
        
#         # Hidden layer
#         h1 = tf.layers.dense(inputs, n_units, activation=None)
#         print('hidden is:', h1)
        
#         # Leaky ReLU
#         h1 = tf.maximum(alpha * h1, h1)
#         print('hidden is:', h1)

#         logits = tf.layers.dense(h1, 1, activation=None)
#         print('logits is:', logits)
#         out = tf.sigmoid(logits)
#         print('discriminator out is:', out)
        
#         return out, logits

In [0]:
def encoder(image_input, latent_size, n_units=128, reuse=False, alpha=0.01, scope='encoder'):
    with tf.variable_scope(scope, reuse=reuse):
        print('inside encoder')
#         input_ = Input(shape=IMAGE_SHAPE)
        input_ = tf.reshape(image_input, shape=[-1, 64, 64, 3])
        print('input_ is:', input_)
        x = input_
        print('x is:', x)
        x = conv_block(x,128)
        print('x is:', x)
        x = conv_block(x,256)
        print('x is:', x)
        x = conv_block(x,512)
        print('x is:', x)
        x = conv_block(x,1024)
        print('x is:', x)
        x = Dense(ENCODER_DIM)(Flatten()(x))
        print('x is:', x)
        x = Dense(4 * 4 * 1024)(x)
        print('x is:', x)
        x = Reshape((4, 4, 1024))(x)
        print('x is:', x)
        x = upscale_block(x,512)
        print('x is:', x)
        
        return x
        
#         # Hidden layer
#         hidden = tf.layers.dense(image_input, n_units, activation=None) # 160 -> 128
        
#         # Leaky ReLU
#         hidden = tf.maximum(hidden, alpha*hidden)
        
#         # Logits and tanh output
#         x = tf.layers.dense(x, latent_size, activation=None) # 128 -> 100
#         latents = tf.nn.tanh(x)
#         return latents
    
def decoder(latents, out_dim, n_units=128, reuse=False, alpha=0.01, scope='decoder'):
    with tf.variable_scope(scope, reuse=reuse):
        print('inside decoder')
        input_ = Input(shape=(8, 8, 512))
        input_ = tf.reshape(latents, shape=[-1, 8, 8, 512])
        print('input_ is:', input_)
        x = input_
        print('x is:', x)
#         x = latents
        x = upscale_block(x,256)
        print('x is:', x)
        
#         # Hidden layer
#         x = tf.layers.dense(latents, n_units, activation=None) # 100 -> 128
#         print('x shape is:', x.shape)
        x = upscale_block(x,128)
        print('x is:', x)
        x = upscale_block(x,64)
        print('x is:', x)
        x = Conv2D(3, kernel_size=5, padding='same', activation='sigmoid')(x)
        print('x is:', x)


    
        x = Dense(out_dim)(Flatten()(x))
        print('x is:', x)
        x = Dense(64 * 64 * 3)(x)
        print('x shape is:', x.shape)
#         x = Reshape((64, 64, 3))(x)
#         print('x shape is:', x.shape)
        
#         x = tf.reshape(latents, shape=[-1, 12288])
#         print('will leave the decoder, x shape is : ' , x)

#         logits = tf.layers.dense(x, out_dim, activation=None) # 128 -> out_dim = 160 || 64
        x = tf.nn.tanh(x)
      
        return x
       
#         # Leaky ReLU
#         hidden = tf.maximum(hidden, alpha*hidden)

#         Logits and tanh output
#         logits = tf.layers.dense(x, out_dim, activation=None) # 128 -> out_dim = 160 || 64
#         tanh_out = tf.nn.tanh(logits)
        
#         return tanh_out
    
def discriminator(inputs, n_units=128, reuse=False, alpha=0.01, scope='discriminator'):
    print('inside discriminator model function')
    activation = lrelu
    with tf.variable_scope(scope, reuse=reuse):
        
#         # Hidden layer
#         print('inputs is: ' , inputs) 
#         print('n_units is: ' , n_units)
        
#         hidden = tf.layers.dense(inputs, n_units, activation=None) # out_dim = 160 || 64 -> 128 
#         print(hidden)
#         # Leaky ReLU
#         hidden = tf.maximum(hidden, alpha*hidden)

        x = tf.reshape(inputs, shape=[-1, 64, 64, 3])
        print('x is:', x)
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=2, padding='same', activation=activation)
        print('x is:', x)
        x = tf.layers.dropout(x,0.6)
        print('x is:', x)
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        print('x is:', x)
        x = tf.layers.dropout(x,0.6)
        print('x is:', x)
        x = tf.layers.conv2d(x, kernel_size=5, filters=64, strides=1, padding='same', activation=activation)
        print('x is:', x)
        x = tf.layers.dropout(x, 0.6)
        print('x is:', x)
        x = tf.contrib.layers.flatten(x)
        print('x is:', x)
        x = tf.layers.dense(x, units=128, activation=activation)
        print('x is:', x)
#         sigmoid_out = tf.layers.dense(x, units=1, activation=tf.nn.sigmoid)

        # Sigmoid output and Logits 
        logits = tf.layers.dense(x, 1, activation=None) # 128 -> 1
        print('logits:', logits)
        sigmoid_out = tf.sigmoid(logits)
        print('sigmoid_out:', sigmoid_out )
        
        return sigmoid_out, logits

In [0]:
def train(imageset_a, imageset_b, input_size, latent_size, g_hidden_size, d_hidden_size, 
          alpha, smooth, learning_rate, batch_size, epochs, reuse):
    # Build Network
    tf.reset_default_graph()
    
    # Necessary Scopes
    generator_encoder_scope = 'generator'
    generator_decoder_a_scope = 'dec_a'
    generator_decoder_b_scope = 'dec_b'
    discriminator_a_scope = 'discriminator_a'
    discriminator_b_scope = 'discriminator_b'
    
#     print('default graph and scope seting finish.')
#     print("imageset a shape is: " + str(imageset_a.shape) + ";imageset b shape is: " + str(imageset_b.shape))
    
    
    with tf.device('/device:GPU:0'):
      # Create our input placeholders
      input_real, input_fake = model_inputs(input_size)
      
      # Build the model
      # 1.g_model is the generator output
      g_model_enc = encoder(input_fake, latent_size, n_units=g_hidden_size, 
                            alpha=alpha, scope=generator_encoder_scope)
      g_model_dec_a = decoder(g_model_enc, input_size, n_units=g_hidden_size, 
                              alpha=alpha, scope=generator_decoder_a_scope)
      g_model_dec_b = decoder(g_model_enc, input_size, n_units=g_hidden_size, 
                              alpha=alpha, scope=generator_decoder_b_scope)
      
#       print('g_model seting finish.')
      
      # 2.d_model is the discrimator output
      d_model_real_a, d_logits_real_a = discriminator(input_real, n_units=d_hidden_size, 
                                                      alpha=alpha, scope=discriminator_a_scope)
      d_model_fake_a, d_logits_fake_a = discriminator(g_model_dec_a, reuse=True, n_units=d_hidden_size, 
                                                      alpha=alpha, scope=discriminator_a_scope)
      d_model_real_b, d_logits_real_b = discriminator(input_real, n_units=d_hidden_size, 
                                                      alpha=alpha, scope=discriminator_b_scope)
      d_model_fake_b, d_logits_fake_b = discriminator(g_model_dec_b, reuse=True, n_units=d_hidden_size, 
                                                      alpha=alpha, scope=discriminator_b_scope)
      
#       print('d_model seting finish.')
      
      # Define the loss functions, get the trainable_variables, split into G and D parts
      d_loss_real_a = tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real_a, 
                                                                labels=tf.ones_like(d_logits_real_a) * (1 - smooth)))
      d_loss_fake_a = tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake_a, 
                                                                labels=tf.zeros_like(d_logits_fake_a)))
      d_loss_a = d_loss_real_a + d_loss_fake_a
      g_loss_a = tf.reduce_mean(
                   tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake_a,
                                                           labels=tf.ones_like(d_logits_fake_a)))
      d_loss_real_b = tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real_b, 
                                                                labels=tf.ones_like(d_logits_real_b) * (1 - smooth)))
      d_loss_fake_b = tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake_b, 
                                                                labels=tf.zeros_like(d_logits_fake_b)))
      d_loss_b = d_loss_real_b + d_loss_fake_b
      g_loss_b = tf.reduce_mean(
                   tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake_b,
                                                           labels=tf.ones_like(d_logits_fake_b)))
      
#       print('loss seting finish.')
      
      t_vars = tf.trainable_variables()
      g_vars = [var for var in t_vars if var.name.startswith(generator_encoder_scope)]
      g_vars_a = [var for var in t_vars if var.name.startswith(generator_decoder_a_scope)]
      d_vars_a = [var for var in t_vars if var.name.startswith(discriminator_a_scope)]
      g_vars_b = [var for var in t_vars if var.name.startswith(generator_decoder_b_scope)]
      d_vars_b = [var for var in t_vars if var.name.startswith(discriminator_b_scope)]
      
      d_train_opt_a = tf.train.AdamOptimizer(learning_rate).minimize(d_loss_a, var_list=d_vars_a)
      g_train_opt_a = tf.train.AdamOptimizer(learning_rate).minimize(g_loss_a, var_list=g_vars + g_vars_a)
      d_train_opt_b = tf.train.AdamOptimizer(learning_rate).minimize(d_loss_b, var_list=d_vars_b)
      g_train_opt_b = tf.train.AdamOptimizer(learning_rate).minimize(g_loss_b, var_list=g_vars + g_vars_b)
      
#       print('Adam opt finish.')
    
    samples_a = []
    samples_b = []
    losses = []
    # Only save generator variables
    saver = tf.train.Saver(var_list=g_vars + g_vars_a + g_vars_b + d_vars_a+ d_vars_b)
    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
#         sess.run(tf.global_variables_initializer())
#         sess.run(tf.local_variables_initializer())
        init_op = tf.group(tf.initialize_all_variables(), tf.initialize_local_variables())
        sess.run(init_op)
        if reuse:
            print("Reloading from checkpoint")
            saver = tf.train.import_meta_graph('/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code/checkpoints/GAN_b.ckpt.meta')
            saver.restore(sess, '/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code/checkpoints/GAN_b.ckpt')
        for e in range(epochs):
          
            print('Epochs begin at', e)
            
            
            np.random.shuffle(imageset_a)
            np.random.shuffle(imageset_b)
            
#             print("imageset a shape is: " + str(imageset_a.shape) + ";imageset b shape is: " + str(imageset_b.shape))
            
            def epoch_optimizer(imageset_real, imageset_fake, train_opt, loss):               
                for ii in range(min(imageset_real.shape[0]//batch_size, imageset_fake.shape[0]//batch_size)):
                    
#                     print('epoch_optimizer begin at', ii  )
                    
                    batch_real = np.array(imageset_real[ii*batch_size:(ii+1)*batch_size])
                    batch_fake = np.array(imageset_fake[ii*batch_size:(ii+1)*batch_size])
                    
#                     print('epoch_optimizer 1:', ii)
#                     print("batch_real shape is: " + str(batch_real.shape) + ";batch_fake shape is: " + str(batch_fake.shape))
                    
                    
                    # Get images, reshape and rescale to pass to D
                    batch_images_real = batch_real.reshape((batch_size, input_size))
                    
#                     print('epoch_optimizer 2:', ii)
#                     print("batch_images_real shape is: " + str(batch_images_real.shape))
                    
                    
                    # Adversarial images for G
                    batch_images_fake = batch_fake.reshape((batch_size, input_size))
                    
#                     print('epoch_optimizer 3:', ii)
#                     print("batch_images_fake shape is: " + str(batch_images_fake.shape))
#                     print("train_opt:", train_opt)
#                     print("feed dict:", {input_real: batch_images_real,input_fake: batch_images_fake})
                    
                    # Run optimizers
                    _ = sess.run(train_opt, feed_dict={input_real: batch_images_real, 
                                                         input_fake: batch_images_fake})
                    
                    
#                     print('train_opt: ', train_opt)
#                     feed_dict_= {input_real: batch_images_real, input_fake: batch_images_fake}
#                     print('feed_dict_: ', feed_dict_)
                                 
#                     print('epoch_optimizer 4:', ii)
                      
                # At the end of each epoch, get the losses and print them out
                train_loss = sess.run(loss, {input_fake: imageset_fake, input_real: imageset_real})
                
#                 print('loss now: ', loss)
#                 feed_dict_tloss = {input_real: batch_images_real, input_fake: batch_images_fake}
#                 print('feed_dict_tloss: ', feed_dict_tloss)
                
#                 print('epoch_optimizer 5:', ii)
                  
                return train_opt, train_loss
            
#             print('start epoch_optimizer')
            
            d_train_opt_a, train_loss_d_a = epoch_optimizer(imageset_a, imageset_b, d_train_opt_a, d_loss_a)
            
#             print('d_train_opt_a, train_loss_d_a finish.')
            
            d_train_opt_b, train_loss_d_b = epoch_optimizer(imageset_b, imageset_a, d_train_opt_b, d_loss_b)
            
#             print('d_train_opt_b, train_loss_d_b finish.')
              
            g_train_opt_a, train_loss_g_a = epoch_optimizer(imageset_a, imageset_b, g_train_opt_a, g_loss_a)
            
#             print('g_train_opt_a, train_loss_g_a finish.')
            
            
            g_train_opt_b, train_loss_g_b = epoch_optimizer(imageset_b, imageset_a, g_train_opt_b, g_loss_b)
            
#             print('g_train_opt_b, train_loss_g_b finish.')
            
#             print('finish epoch_optimizer')
            
            # Statements to print each epoch
               

#             if (e+1)%50 == 0 or (e == 0):
#                 print("Epoch {}/{}...".format(e+1, epochs),
#                       "Discriminator_a Loss: {:.4f}...".format(train_loss_d_a),
#                       "Generator_a Loss: {:.4f}".format(train_loss_g_a),
#                       "Discriminator_b Loss: {:.4f}...".format(train_loss_d_b),
#                       "Generator_b Loss: {:.4f}".format(train_loss_g_b))
            
            # Save losses to view after training
            losses = (train_loss_d_a, train_loss_g_a, train_loss_d_b, train_loss_g_b)
            
            print('losses da,ga,db,gb saved as:', losses)
            
            # Sample from generator as we're training for viewing afterwards
            if (e+1) == epochs:
                
                batch_a = np.array(imageset_a[0:batch_size])
                batch_b = np.array(imageset_b[0:batch_size])
                
                print('batch_a', batch_a.shape)
                print('batch_b', batch_b.shape)
                
#                 sess.run(tf.local_variables_initializer())
                encoder_val = encoder(input_fake, latent_size, n_units=g_hidden_size, alpha=alpha, reuse=True, 
                                scope=generator_encoder_scope)
                
                print('encoder_val', encoder_val)
                
#                 sess.run(tf.local_variables_initializer())
                decoder_a_val = decoder(encoder_val, input_size, n_units=g_hidden_size, 
                                         reuse=True, scope=generator_decoder_a_scope)
                
                print('decoder_a_val', decoder_a_val)
                feed_dict_decoder= {input_fake: batch_b}
                
                print('feed_dict_decoder', feed_dict_decoder)
                
                
#                 sess.run(tf.local_variables_initializer())
                gen_samples_a = sess.run(decoder_a_val, feed_dict_decoder)
                
                print('samples_a saved as:', samples_a)
                
                gen_samples_b = sess.run(
                               decoder(encoder(input_fake, latent_size, n_units=g_hidden_size, alpha=alpha, reuse=True, 
                                scope=generator_encoder_scope), input_size, n_units=g_hidden_size, reuse=True,
                                scope=generator_decoder_b_scope), feed_dict={input_fake: batch_a})
                
                print('samples_b saved as:', samples_b)
                
                samples_a = gen_samples_a
                samples_b = gen_samples_b

            if e != 0 and e % 5 == 0:
                save_loc = saver.save(sess, '/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code/checkpoints/GAN_b.ckpt')
        
        # This change will solve the memory problem
        #save_loc = saver.save(sess, '/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code/checkpoints/GAN.ckpt')
    
    # Save training generator samples
    with open('/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code/train_samples_a.pkl', 'wb') as f:
        pkl.dump(samples_a, f)
    with open('/content/gdrive/My Drive/UIUC/2019_spring/ECE549--computer vision/Project/code/train_samples_b.pkl', 'wb') as f:
        pkl.dump(samples_b, f)
        
    return samples_a, samples_b, losses, g_vars, g_vars_a, g_vars_b

In [30]:
# SETUP IMAGES HERE, Images is array [image number, size of image]


img_dirA = 'faceA/rgb'
img_dirB = 'faceB/rgb'
train_A = glob.glob(img_dirA+"/*.*")
train_B = glob.glob(img_dirB+"/*.*")
# images_a = np.array([np.array(Image.open(fname).resize((160,160))) for fname in train_a])
# images_b = np.array([np.array(Image.open(fname).resize((160,160))) for fname in train_b])

images_a = np.array([np.array(Image.open(fname).resize((64,64))) for fname in train_a])
images_b = np.array([np.array(Image.open(fname).resize((64,64))) for fname in train_b])

print("images a shape is: " + str(images_a.shape) + ";images b shape is: " + str(images_b.shape))


flattendedimages_a = np.array(images_a).reshape((images_a.shape[0], -1))
flattendedimages_b = np.array(images_b).reshape((images_b.shape[0], -1))
# Size of input images to discriminators (SIZE OF IMAGE)
assert(flattendedimages_a.shape[1] == flattendedimages_b.shape[1])
input_size = flattendedimages_a.shape[1]

print("input size is: " + str(flattendedimages_a.shape[1]))


# Scale images to fit functions
scaledimages_a = preprocess_image(flattendedimages_a)
scaledimages_b = preprocess_image(flattendedimages_b)

# Size of latent vector to generator
latent_size = 100
# Sizes of hidden layers in generator and discriminator
g_hidden_size = int(math.pow(2, int(math.log(input_size, 2)) - int(math.log(latent_size * 2, 2))))
d_hidden_size = g_hidden_size
# Leak factor for leaky ReLU
alpha = 0.01
# Smoothing 
smooth = 0.1 # used for loss defination
# Optimizers
learning_rate = 0.001
# Step size
batch_size = 100
# Training Rounds
epochs = 2 # Having run around 200 + 1500 + 2500 + 800 + 2100 + 2900
# If we want to reuse old model
reuse = False

samples_a, samples_b, losses, g_vars, g_vars_a, g_vars_b = train(scaledimages_a, scaledimages_b, input_size, 
                latent_size, g_hidden_size, d_hidden_size, alpha, smooth, learning_rate, batch_size, epochs, reuse)

images a shape is: (708, 64, 64, 3);images b shape is: (647, 64, 64, 3)
input size is: 12288
inside encoder
input_ is: Tensor("generator/Reshape:0", shape=(?, 64, 64, 3), dtype=float32, device=/device:GPU:0)
x is: Tensor("generator/Reshape:0", shape=(?, 64, 64, 3), dtype=float32, device=/device:GPU:0)
Instructions for updating:
Colocations handled automatically by placer.
x is: Tensor("generator/leaky_re_lu_1/LeakyRelu:0", shape=(?, 32, 32, 128), dtype=float32, device=/device:GPU:0)
x is: Tensor("generator/leaky_re_lu_2/LeakyRelu:0", shape=(?, 16, 16, 256), dtype=float32, device=/device:GPU:0)
x is: Tensor("generator/leaky_re_lu_3/LeakyRelu:0", shape=(?, 8, 8, 512), dtype=float32, device=/device:GPU:0)
x is: Tensor("generator/leaky_re_lu_4/LeakyRelu:0", shape=(?, 4, 4, 1024), dtype=float32, device=/device:GPU:0)
x is: Tensor("generator/dense_1/BiasAdd:0", shape=(?, 1024), dtype=float32, device=/device:GPU:0)
x is: Tensor("generator/dense_2/BiasAdd:0", shape=(?, 16384), dtype=float32, d

FailedPreconditionError: ignored

In [0]:
img = samples_a[-1]
# img = postprocess_image(flattendedimages_a, img.reshape(160,160,3))
img = postprocess_image(flattendedimages_a, img.reshape(64,64,3))
imgplot = plt.imshow(img)
plt.show()

In [0]:
img = samples_b[-1]
# img = postprocess_image(flattendedimages_b, img.reshape(160,160,3))
img = postprocess_image(flattendedimages_b, img.reshape(64,64,3))
imgplot = plt.imshow(img)
plt.show()